# Transformers

<div class="alert  alert-warning" role="alert">
    Note this notebook should be run using the <strong>nlp-1.2 kernel</strong> on HiPerGator.
</div>
    
The goal of this notebook is to try to make the links from NLP to the emergence of transformer architectures in NLP to set us up for looking at broader applications of transformers beyond NLP. I'm trying to not get bogged down in the details of NLP or transformers, but provide some intuition on how transformers revolutionized NLP and how they are now moving on to impact other fields.

In his [GTC 2022 Keynote](https://www.nvidia.com/gtc/keynote/), Nvidia's CEO, Jensen  Huang, stated:
> The Transformer is unquestionably the most important deep learning model invented.

That says a lot for an architecture introduced only 5-years earlier in 2017!

This notebook largely follows, and quotes from, the [Hugging Face Transformer Course](https://huggingface.co/course/chapter1/3?fw=tf). It starts with some motivating examples.

Here are some examples of what Transformers can do in NLP:

## Set up Cache directory

By default, the Huggingface `transformers` module will download things to `~/.chache/huggingface`. There are a couple of problems with that. First, HiPerGator home directories are limited to 40GB and these files can add up fast. Second, it can take a while to download the first time you run. So, I've downloaded everything for this notebook and copied the cache to `/blue/zoo4926/share/huggingface/`. Let's link your cache to there.

Note...this might not be the best method. But I ran out of time and the documentation didn't seem to match actual behavior for changing the default path... I'll provide the cell to undo the change too...

In [3]:
# Make a symbolic link in your ~/.cache folder to /blue/zoo4926/share/huggingface/

!ln -s /blue/zoo4926/share/huggingface/ ~/.cache/huggingface

**ONLY RUN THE FOLLOWING CELL WHEN YOU ARE DONE WITH THE NOTEBOOK**

When you are done with this notebook, you can remove this link.

Uncomment (remove the #) the line below an drun the cell

In [4]:
# Uncomment and run when done with this notebook.
#!rm ~/.cache/huggingface

## Some Examples

### Sentiment Analysis

In [2]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier("I've been waiting for a HuggingFace course my whole life.")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


[{'label': 'POSITIVE', 'score': 0.9598047137260437}]

> By default, this pipeline selects a particular pre-trained model that has been fine-tuned for sentiment analysis in English. The model is downloaded and cached when you create the `classifier` object. If you rerun the command, the cached model will be used instead and there is no need to download the model again.
>
> There are three main steps involved when you pass some text to a pipeline:
>
> 1. The text is preprocessed into a format the model can understand.
> 1. The preprocessed inputs are passed to the model.
> 1. The predictions of the model are post-processed, so you can make sense of them.
>
> Some of the currently [available pipelines](https://huggingface.co/transformers/main_classes/pipelines.html) are:
> 
> * `feature-extraction` (get the vector representation of a text)
> * `fill-mask`
> * `ner` (named entity recognition)
> * `question-answering`
> * `sentiment-analysis`
> * `summarization`
> * `text-generation`
> * `translation`
> * `zero-shot-classification`
>
>Let’s have a look at a few of these!

### Zero-shot classification

> We’ll start by tackling a more challenging task where we need to classify texts that haven’t been labelled. This is a common scenario in real-world projects because annotating text is usually time-consuming and requires domain expertise. For this use case, the zero-shot-classification pipeline is very powerful: it allows you to specify which labels to use for the classification, so you don’t have to rely on the labels of the pre-trained model. You’ve already seen how the model can classify a sentence as positive or negative using those two labels — but it can also classify the text using any other set of labels you like.

In [3]:
classifier = pipeline("zero-shot-classification")
classifier(
    "This is a course about the Transformers library",
    candidate_labels=["education", "politics", "business"],
)

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)


Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

{'sequence': 'This is a course about the Transformers library',
 'labels': ['education', 'business', 'politics'],
 'scores': [0.8445965647697449, 0.11197599768638611, 0.04342741146683693]}

> This pipeline is called zero-shot because you don’t need to fine-tune the model on your data to use it. It can directly return probability scores for any list of labels you want!

### Text Generation

> Now let’s see how to use a pipeline to generate some text. The main idea here is that you provide a prompt and the model will auto-complete it by generating the remaining text. This is similar to the predictive text feature that is found on many phones. Text generation involves randomness, so it’s normal if you don’t get the same results as shown below.

**Note** The default `gpt2` model does not seem to work on HiPerGator at this time.

In [6]:
generator = pipeline("text-generation", model="distilgpt2")
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=2,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to learn more skills of communication in writing. Topics include: Communication and writing writing\n\n\n\n\n'},
 {'generated_text': 'In this course, we will teach you how to build an efficient web server based on your users base and your user base. The first part will show'}]

### Mask Filling

In [8]:
unmasker = pipeline("fill-mask")
unmasker("This course will teach you all about <mask> models.", top_k=2)

No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)


Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

[{'sequence': 'This course will teach you all about mathematical models.',
  'score': 0.1961984932422638,
  'token': 30412,
  'token_str': ' mathematical'},
 {'sequence': 'This course will teach you all about computational models.',
  'score': 0.04052729159593582,
  'token': 38163,
  'token_str': ' computational'}]

### Named Entity Recognition (NER)

In [9]:
ner = pipeline("ner", grouped_entities=True)
ner("My name is Matt and I work at the University of Florida in Gainesville.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english)
/apps/nlp/1.2/lib/python3.8/site-packages/transformers/pipelines/token_classification.py:128: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


[{'entity_group': 'PER',
  'score': 0.99930125,
  'word': 'Matt',
  'start': 11,
  'end': 15},
 {'entity_group': 'ORG',
  'score': 0.997988,
  'word': 'University of Florida',
  'start': 34,
  'end': 55},
 {'entity_group': 'LOC',
  'score': 0.9874652,
  'word': 'Gainesville',
  'start': 59,
  'end': 70}]

### Question answering

In [11]:
question_answerer = pipeline("question-answering")
question_answerer(
    question="Where do I work?",
    context="My name is Matt and I work at the University of Florida in Gainesville.",
)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


{'score': 0.4767897427082062,
 'start': 34,
 'end': 55,
 'answer': 'University of Florida'}

### Summarization

In [13]:
summarizer = pipeline("summarization")
summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of 
    graduates in traditional engineering disciplines such as mechanical, civil, 
    electrical, chemical, and aeronautical engineering declined, but in most of 
    the premier American universities engineering curricula now concentrate on 
    and encourage largely the study of engineering science. As a result, there 
    are declining offerings in engineering subjects dealing with infrastructure, 
    the environment, and related issues, and greater concentration on high 
    technology subjects, largely supporting increasingly complex scientific 
    developments. While the latter is important, it should not be at the expense 
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other 
    industrial countries in Europe and Asia, continue to encourage and advance 
    the teaching of engineering. Both China and India, respectively, graduate 
    six and eight times as many traditional engineers as does the United States. 
    Other industrial countries at minimum maintain their output, while America 
    suffers an increasingly serious decline in the number of engineering graduates 
    and a lack of well-educated engineers.
"""
)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


[{'summary_text': ' America has changed dramatically during recent years . The number of engineering graduates in the U.S. has declined in traditional engineering disciplines such as mechanical, civil,    electrical, chemical, and aeronautical engineering . Rapidly developing economies such as China and India continue to encourage and advance the teaching of engineering .'}]

### Translation

In [14]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")
translator("Ce cours est produit par Hugging Face.")


Downloading:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/784k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.28M [00:00<?, ?B/s]

[{'translation_text': 'This course is produced by Hugging Face.'}]

## Tranformers transform NLP and more!

Transformers are not that old! Only having been introduced in 2017! Vaswani *et al.*'s paper [*Attention is all you need*](https://arxiv.org/abs/1706.03762) has over 38,000 citations in Google Scholar (3/22/2022)

![Transformer timeline from Hugging Face](images/transformers_chrono_HuggingFace.png)

Since its publication, transformers have revolutionized NLP and been applied to other fields, including computer vision.

### Transformers are language models

> All the Transformer models mentioned above (GPT, BERT, BART, T5, etc.) have been trained as language models. This means they have been trained on large amounts of raw text in a self-supervised fashion. **Self-supervised learning is a type of training in which the objective is automatically computed from the inputs of the model. That means that humans are not needed to label the data!**
> 
> This type of model develops a statistical understanding of the language it has been trained on, but it’s not very useful for specific practical tasks. Because of this, the general pre-trained model then goes through a process called transfer learning. During this process, the model is fine-tuned in a supervised way — that is, using human-annotated labels — on a given task.

### Transformers are **BIG** models

The graph below is from the article [*We Might See A 100T Language Model In 2022*](https://analyticsindiamag.com/we-might-see-a-100t-language-model-in-2022/), published in Dec 2021:
![Image of transformer model size over time, from https://analyticsindiamag.com/we-might-see-a-100t-language-model-in-2022/](images/NVIDIA_NLP_Model_Size.png)

And costly in terms of compute and CO2 emissions...

![Relative CO2 emissions for a variety of human activities, from Hugging Face](images/carbon_footprint_HuggingFace.png)

Luckily, as we've seen, transfer learning can help!!

![Schematic of pre-training from Hugging Face](images/pretraining_HuggingFace.png)

> This pre-training is usually done on very large amounts of data. Therefore, it requires a very large corpus of data, and training can take up to several weeks.
>
> *Fine-tuning*, on the other hand, is the training done **after** a model has been pre-trained. To perform fine-tuning, you first acquire a pre-trained language model, then perform additional training with a dataset specific to your task.

## Transformer architecture

> The model is primarily composed of two blocks:
> * **Encoder (left)**: The encoder receives an input and builds a representation of it (its features). This means that the model is optimized to acquire understanding from the input.
> * **Decoder (right)**: The decoder uses the encoder’s representation (features) along with other inputs to generate a target sequence. This means that the model is optimized for generating outputs.

![Basic transformer architecture, from Hugging Face](images/transformers_blocks_HuggingFace.png)

> Each of these parts can be used independently, depending on the task:
> * **Encoder-only models**: Good for tasks that require understanding of the input, such as sentence classification and named entity recognition.
> * **Decoder-only models**: Good for generative tasks such as text generation.
> * **Encoder-decoder models** or **sequence-to-sequence models**: Good for generative tasks that require an input, such as translation or summarization.

## Attention layers

> A key feature of Transformer models is that they are built with special layers called attention layers. In fact, the title of the paper introducing the Transformer architecture was ["Attention Is All You Need"](https://arxiv.org/abs/1706.03762)! ...for now, all you need to know is that this layer will tell the model to pay specific attention to certain words in the sentence you passed it (and more or less ignore the others) when dealing with the representation of each word.
>
> To put this into context, consider the task of translating text from English to French. Given the input “You like this course”, a translation model will need to also attend to the adjacent word “You” to get the proper translation for the word “like”, because in French the verb “like” is conjugated differently depending on the subject. The rest of the sentence, however, is not useful for the translation of that word. In the same vein, when translating “this” the model will also need to pay attention to the word “course”, because “this” translates differently depending on whether the associated noun is masculine or feminine. Again, the other words in the sentence will not matter for the translation of “this”. With more complex sentences (and more complex grammar rules), the model would need to pay special attention to words that might appear farther away in the sentence to properly translate each word.
>
> The same concept applies to any task associated with natural language: a word by itself has a meaning, but that meaning is deeply affected by the context, which can be any other word (or words) before or after the word being studied.
> 
> Now that you have an idea of what attention layers are all about, let’s take a closer look at the Transformer architecture.

## The original architecture

> The Transformer architecture was originally designed for translation. During training, the **encoder** receives inputs (sentences) in a certain language, while the **decoder** receives the same sentences in the desired target language. In the encoder, the attention layers can use all the words in a sentence (since, as we just saw, the translation of a given word can be dependent on what is after as well as before it in the sentence). The decoder, however, works sequentially and can only pay attention to the words in the sentence that it has already translated (so, only the words before the word currently being generated). For example, when we have predicted the first three words of the translated target, we give them to the decoder which then uses all the inputs of the encoder to try to predict the fourth word.
>
> To speed things up during training (when the model has access to target sentences), the decoder is fed the whole target, but it is not allowed to use future words (if it had access to the word at position 2 when trying to predict the word at position 2, the problem would not be very hard!). For instance, when trying to predict the fourth word, the attention layer will only have access to the words in positions 1 to 3.
> 
> The original Transformer architecture looked like this, with the encoder on the left and the decoder on the right:

![Architecture of a Transformers models](images/transformers.png)

> Note that the the first attention layer in a decoder block pays attention to all (past) inputs to the decoder, but the second attention layer uses the output of the encoder. It can thus access the whole input sentence to best predict the current word. This is very useful as different languages can have grammatical rules that put the words in different orders, or some context provided later in the sentence may be helpful to determine the best translation of a given word.
> 
> The attention mask can also be used in the encoder/decoder to prevent the model from paying attention to some special words — for instance, the special padding word used to make all the inputs the same length when batching together sentences.



## Biases and Limiatations of NLP Models

> If your intent is to use a pretrained model or a fine-tuned version in production, please be aware that, while these models are powerful tools, they come with limitations. The biggest of these is that, to enable pretraining on large amounts of data, researchers often scrape all the content they can find, taking the best as well as the worst of what is available on the internet.
>
> To give a quick illustration, let’s go back the example of a fill-mask pipeline with the BERT model:

In [15]:
unmasker = pipeline("fill-mask", model="bert-base-uncased")
result = unmasker("This man works as a [MASK].")
print([r["token_str"] for r in result])

result = unmasker("This woman works as a [MASK].")
print([r["token_str"] for r in result])

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

['carpenter', 'lawyer', 'farmer', 'businessman', 'doctor']
['nurse', 'maid', 'teacher', 'waitress', 'prostitute']
